In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import mediapipe as mp

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Function to load and preprocess images with Mediapipe
def load_and_preprocess_images(directory, label):
    images = []
    labels = []

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = cv2.imread(os.path.join(directory, filename))
            img = cv2.resize(img, (64, 64))

            # Use Mediapipe to extract hand landmarks or other information
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            # Modify this part based on your specific use case with Mediapipe
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Process hand landmarks as needed
                    # Example: hand_landmarks.landmark contains the 3D coordinates of hand landmarks

            images.append(img)
            labels.append(label)

    hands.close()  # Close the mediapipe hands module
    return np.array(images), np.array(labels)

# Load and preprocess fall and non-fall images with Mediapipe
fall_images, fall_labels = load_and_preprocess_images("/content/drive/MyDrive/Dataset/fall_dataset", label=1)
non_fall_images, non_fall_labels = load_and_preprocess_images("/content/drive/MyDrive/Dataset/non_fall_dataset", label=0)

# Concatenate fall and non-fall images and labels
all_images = np.concatenate([fall_images, non_fall_images], axis=0)
all_labels = np.concatenate([fall_labels, non_fall_labels], axis=0)

# Shuffle the dataset
indices = np.arange(all_images.shape[0])
np.random.shuffle(indices)
all_images = all_images[indices]
all_labels = all_labels[indices]

# Normalize pixel values to be between 0 and 1
all_images = all_images / 255.0

# Build and compile the model as before
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay), input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, weight_decay=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(all_images, all_labels, epochs=10, batch_size=32, validation_split=0.2)

# Save the trained model to Google Drive
model.save("/content/drive/MyDrive/fall_detection_model.h5")
